# Data Collection

## Objectives:

1. **Load and Preprocess Cheery Leaves Images:**
    - Read and load cherry leaf images from provided dataset.
    - Perform any necessary preprocessing steps, such as resizing and normalization.

2. **Annotate images with Labels:**
    - Assign labels to each image indicating, whether the cherry leaf is healthy or contains powdery milldew.

## Inputs:

1. **Dataset:**
   - A dataset containing cherry leaf images.
   - The dataset should have a structure where images are organized in folders or directories.
   - Each image should be associated with a specific class or label (healthy or mildew).

2. **Paths:**
   - Path to the local dataset on your computer (`dataset_path`).
   - Path to the Gitpod workspace (`workspace_path`).

## Outputs:

1. **Processed Images:**
   - The preprocessed cherry leaf images ready for training.

2. **Labels:**
   - A mapping between image filenames and their corresponding labels (healthy or mildew).

## Additional Comments:

   - no additional comments




---

# Import packages

In [ ]:
%pip install -r /workspace/Portfolio-project-5-Milldew-detection-in-Cherry-Leaves/requirements.txt

In [ ]:
import os
import numpy

### Change the working directory

In [ ]:
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir('/workspace/Portfolio-project-5-Milldew-detection-in-Cherry-Leaves/notebooks')
print("You set a new current directory")

In [ ]:
current_dir = os.getcwd()
current_dir

### Install Kaggle

In [ ]:
# install kaggle package
%pip install kaggle==1.5.12

---

### Set Kaggle API key path

In [ ]:
kaggle_key_path = '/workspace/Portfolio-project-5-Milldew-detection-in-Cherry-Leaves/kaggle.json'

### Set Kaggle environment variable

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(kaggle_key_path)

### Change permissions

In [ ]:
! chmod 600 {kaggle_key_path}

Set kaggle dataset and download it

In [ ]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry_leaves_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Unzip the downloaded file and delete the zip file

In [ ]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data preparation

---

## Data cleaning

### Check and remove non-image files

In [ ]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)  # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

In [ ]:
remove_non_image_file(my_data_dir='inputs/cherry_leaves_dataset/cherry-leaves')

### Split train validation test set

In [ ]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)


In [ ]:
split_train_validation_test_images(my_data_dir=f"inputs/cherry_leaves_dataset/cherry-leaves/",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )

---